In [4]:
%config Application.log_level='INFO'
import logging
logging.getLogger().setLevel(logging.INFO)
log = logging.getLogger()
#for hdlr in log.handlers[:]:  # remove all old handlers
#    log.removeHandler(hdlr)
fileh = logging.FileHandler('/storage/vbsjjlnu/log.txt', 'a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fileh.setFormatter(formatter)
log.addHandler(fileh)

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [2]:
import model_opt

Using TensorFlow backend.


In [5]:
config = {
        "base_dir":        "/storage/vbsjjlnu",
        "plot_config":     "Full2018v6s5",
        "cut":             "boost",
        "samples_version": "v10",
        "cols": ['mjj_vbs', 'vbs_0_pt', 'vbs_1_pt', 'deltaeta_vbs', 'deltaphi_vbs', 
            'mjj_vjet', 'vjet_0_pt', 'vjet_1_pt', 'vjet_0_eta', 'vjet_1_eta', 
            'Lepton_pt', 'Lepton_eta', 'Lepton_flavour', 
            'PuppiMET', 
            'Zvjets_0', 'Zlep', 
            'Asym_vbs', 'Asym_vjet', 'A_ww', 
            'Mtw_lep', 'w_lep_pt', 'Mww', 'R_ww', 'R_mw', 
            'Centr_vbs', 'Centr_ww'
            ],
    "verbose": 0
    }
ev = model_opt.test_vbsdnn_model(config)
print(ev)

0.11684126844902726


<Figure size 432x288 with 0 Axes>

## GPyOpt

In [18]:
# !pip3 install --user gpyopt

import GPyOpt

In [83]:
config = {
    "base_dir":        "/storage/vbsjjlnu",
    "plot_config":     "Full2018v6s5",
    "cut":             "boost",
    "samples_version": "v10",
    "cols": ['mjj_vbs','deltaeta_vbs','Centr_ww','Asym_vbs','A_ww',
             'Lepton_eta','mjj_vjet','vbs_0_pt','vbs_1_pt','vjet_0_pt',
             'Lepton_pt','Centr_vbs','w_lep_pt','Mtw_lep','deltaphi_vbs',
             'Mww','PuppiMET','Lepton_flavour','R_mw','vjet_0_eta','Zlep',
             'Zvjets_0','R_ww',
        ],
    "verbose": 0
}
    
bounds = [{'name': 'batch_size', 'type': 'discrete',  'domain': (1024, 2048, 4096)},
          {'name': 'n_layers', 'type': 'discrete',  'domain': (2,3,4,5,6)},
          {'name': 'n_nodes', 'type': 'discrete',  'domain': (10,30,50,80,100,150,200)},
          {'name': 'dropout', 'type': 'discrete',  'domain': (0,0.05,0.1,0.2,0.3)},
          {'name': 'batch_norm', 'type': 'discrete',  'domain': (0,1)},
          {'name': 'input_dim', 'type': 'discrete',  'domain': tuple(list(range(4,len(config["cols"])+1) ))},
         ]


fixed_params={
    "epochs": 200,
    "val_ratio": 0.25,
    "test_ratio": 0.1,
    "patience": (0.001, 10)
}

In [84]:
## optimizer function
def f(x):
    ev = model_opt.evaluate_vbsdnn_model(config, fixed_params, x)
    return ev

In [85]:
opt_mnist = GPyOpt.methods.BayesianOptimization(f=f, domain=bounds)

INFO:root:> L:2 , N:80, BS:4096, D:0.30, BN:True, I:14
INFO:root:1581783129
INFO:root:Start Training
INFO:root:Saving training metadata
INFO:root:Reporting Significance: 0.15219973645591836
INFO:root:> L:5 , N:50, BS:2048, D:0.10, BN:False, I:6
INFO:root:1581783155
INFO:root:Start Training
INFO:root:Saving training metadata
INFO:root:Reporting Significance: 0.14645825803345874
INFO:root:> L:3 , N:30, BS:1024, D:0.00, BN:False, I:9
INFO:root:1581783190
INFO:root:Start Training
INFO:root:Saving training metadata
INFO:root:Reporting Significance: 0.15099210700999735
INFO:root:> L:3 , N:50, BS:2048, D:0.00, BN:False, I:13
INFO:root:1581783220
INFO:root:Start Training
INFO:root:Saving training metadata
INFO:root:Reporting Significance: 0.15031770276769127
INFO:root:> L:3 , N:30, BS:4096, D:0.30, BN:False, I:23
INFO:root:1581783246
INFO:root:Start Training
INFO:root:Saving training metadata
INFO:root:Reporting Significance: 0.15713873751453747


<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [76]:
opt_mnist.run_optimization(max_iter=3)

error: failed in converting 2nd argument `b' of _flapack.dpotrs to C/Fortran array

In [87]:
print("""
Optimized Parameters:
\t{0}:\t{1}
\t{2}:\t{3}
""".format(
    bounds[0]["name"],opt_mnist.x_opt[0],
    bounds[1]["name"],opt_mnist.x_opt[1],
))
print("optimized loss: {0}".format(opt_mnist.fx_opt))


Optimized Parameters:
	test_ratio:	0.1
	batch_size:	256.0

optimized loss: 0.5144698499900197
